In [18]:
from pycocotools.coco import COCO
import json
from coco_camera_traps_loader import load_image
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import CocoDetection
import torch.utils.data as data
from PIL import Image
import os

In [19]:
def get_id_dict(coco, ids, anns):
    id_dict = dict.fromkeys(ids)
    id_set = set(ids)
    for i in anns:
        img_id = coco.anns[i]['image_id']
        if img_id in id_set:
            value = id_dict[img_id]
            if value is not None:
                value.append(i)
            else:
                value = []
                value.append(i)
            pair = {img_id: value}
            id_dict.update(pair)
    return id_dict

In [20]:
class CocoDetection(data.Dataset):
    """`MS Coco Detection <http://mscoco.org/dataset/#detections-challenge2016>`_ Dataset.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """

    def __init__(self, root, annFile, transform=None, target_transform=None):
        from pycocotools.coco import COCO
        self.root = root
        self.coco = COCO(annFile)
        #self.ids = list(self.coco.imgs.keys())
        self.ids = [i for i in list(self.coco.imgs.keys()) if self.coco.imgs[i]['season'] == 'S1']
        self.anns = list(self.coco.anns.keys())
        self.id_dict = get_id_dict(self.coco, self.ids, self.anns)
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, target). target is the object returned by ``coco.loadAnns``.
        """
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = self.id_dict[img_id]
        #ann_ids = coco.getAnnIds(imgIds=img_id)
        target = coco.loadAnns(ann_ids)
        print(target)

        #path = coco.loadImgs(img_id)[0]['file_name']
        path = coco.imgs[img_id]['file_name']
        print(f'{self.root}/{path}')

        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target


    def __len__(self):
        return len(self.ids)

In [21]:
path2data = "../../../scratch/zwinzurk/wild/datasets"
path2json = 'lila/SS_S1/SnapshotSerengetiBboxes_20190903.json'

In [22]:
coco_train = CocoDetection(root = path2data, annFile = path2json)

loading annotations into memory...
Done (t=1.11s)
creating index...
index created!


In [17]:
len(coco_train)

12377

In [6]:
loader = iter(coco_train)

In [16]:
x = next(loader)

[{'id': 'YIBf21541155496232', 'category_id': 1, 'image_id': 'S1/P10/P10_R1/S1_P10_R1_PICT0112', 'bbox': [1215.466308593746, 784.1315917968692, 750.5336914062541, 461.25387668610045]}]
../../../scratch/zwinzurk/wild/datasets/S1/P10/P10_R1/S1_P10_R1_PICT0112.JPG


In [8]:
x

(<PIL.Image.Image image mode=RGB size=2048x1536 at 0x2B0AC8A72F10>,
 [{'id': 'g2pAd1533039515988',
   'category_id': 1,
   'image_id': 'S1/D05/D05_R5/S1_D05_R5_PICT1288',
   'bbox': [0, 210.47589111327744, 2047.0, 1225.999999999995]}])

In [23]:
coco = COCO(path2json)

loading annotations into memory...
Done (t=1.22s)
creating index...
index created!


In [27]:
img_ids = list(coco.imgs.keys())

In [28]:
img_ids[:20]

['S1/D05/D05_R5/S1_D05_R5_PICT1288',
 'S6/E03/E03_R2/S6_E03_R2_IMAG1342',
 'S5/E02/E02_R1/S5_E02_R1_IMAG0672',
 'S6/C09/C09_R1/S6_C09_R1_IMAG0070',
 'S2/K03/K03_R2/S2_K03_R2_IMAG0116',
 'S1/K09/K09_R1/S1_K09_R1_PICT5773',
 'S3/E04/E04_R11/S3_E04_R11_IMAG0262',
 'S5/B06/B06_R1/S5_B06_R1_IMAG0325',
 'S2/V13/V13_R1/S2_V13_R1_IMAG0032',
 'S3/I13/I13_R11/S3_I13_R11_IMAG0554',
 'S4/K07/K07_R1/S4_K07_R1_IMAG2168',
 'S4/F12/F12_R1/S4_F12_R1_IMAG0646',
 'S5/P10/P10_R1/S5_P10_R1_IMAG1392',
 'S1/G01/G01_R1/S1_G01_R1_PICT0063',
 'S4/T13/T13_R2/S4_T13_R2_IMAG1875',
 'S5/H02/H02_R3/S5_H02_R3_IMAG0080',
 'S4/K07/K07_R1/S4_K07_R1_IMAG0282',
 'S6/B08/B08_R1/S6_B08_R1_IMAG0722',
 'S3/E05/E05_R11/S3_E05_R11_IMAG0286',
 'S2/D12/D12_R3/S2_D12_R3_PICT0332']

In [25]:
s1 = [i for i in img_ids if coco.imgs[i]['season'] == 'S1']

In [26]:
s1

['S1/D05/D05_R5/S1_D05_R5_PICT1288',
 'S1/K09/K09_R1/S1_K09_R1_PICT5773',
 'S1/G01/G01_R1/S1_G01_R1_PICT0063',
 'S1/H08/H08_R1/S1_H08_R1_PICT0554',
 'S1/G07/G07_R2/S1_G07_R2_PICT0935',
 'S1/I13/I13_R1/S1_I13_R1_PICT0789',
 'S1/E06/E06_R4/S1_E06_R4_PICT0591',
 'S1/F12/F12_R1/S1_F12_R1_PICT0906',
 'S1/P10/P10_R1/S1_P10_R1_PICT0112',
 'S1/F06/F06_R3/S1_F06_R3_PICT0082',
 'S1/I13/I13_R2/S1_I13_R2_PICT0015',
 'S1/R09/R09_R1/S1_R09_R1_PICT0043',
 'S1/K09/K09_R1/S1_K09_R1_PICT5665',
 'S1/O10/O10_R1/S1_O10_R1_PICT1375',
 'S1/J10/J10_R2/S1_J10_R2_PICT0016',
 'S1/O10/O10_R1/S1_O10_R1_PICT0236',
 'S1/J03/J03_R1/S1_J03_R1_PICT2592',
 'S1/K08/K08_R1/S1_K08_R1_PICT1400',
 'S1/K09/K09_R1/S1_K09_R1_PICT5372',
 'S1/G09/G09_R1/S1_G09_R1_PICT0110',
 'S1/F08/F08_R3/S1_F08_R3_PICT3034',
 'S1/H06/H06_R4/S1_H06_R4_PICT1299',
 'S1/H06/H06_R4/S1_H06_R4_PICT1415',
 'S1/Q10/Q10_R1/S1_Q10_R1_PICT0339',
 'S1/H06/H06_R4/S1_H06_R4_PICT1332',
 'S1/H01/H01_R1/S1_H01_R1_PICT0529',
 'S1/H11/H11_R1/S1_H11_R1_PICT5801',
 

In [ ]:
path2json = 'lila/SS_S1/SnapshotSerengetiBboxes_20190903.json'
coco = COCO(path2json)
ids = list(sorted(coco.imgs.keys()))
anns = list(coco.anns.keys())
id_dict = dict.fromkeys(ids)
for i in anns:
    img_id = coco.anns[i]['image_id']
    value = id_dict[img_id]
    if value is not None:
        value.append(i)
    else:
        value = []
        value.append(i)
    pair = {img_id: value}
    id_dict.update(pair)

In [ ]:
for i in ids[:20]:
    print(d[i])

In [ ]:
d[idx]

In [ ]:
coco_train = CocoDetection(root = path2data, annFile = path2json)

In [ ]:
idx = 3
ids = list(sorted(coco.imgs.keys()))
anns = list(coco.anns.keys())
img_id = ids[idx]
print(img_id)
# ann_ids = coco.getAnnIds(imgIds=img_id)
ann_ids = [i for i in anns if coco.anns[i]['image_id'] == img_id]
print(ann_ids)
target = coco.loadAnns(ann_ids)
print(target)

In [ ]:
d = dict.fromkeys(ids, [])

In [ ]:
d.update({'S1/B04/B04_R1/S1_B04_R1_PICT0006': 'hello'})

In [ ]:
d['S1/B04/B04_R1/S1_B04_R1_PICT0006']

In [ ]:
value = d['S1/B04/B04_R1/S1_B04_R1_PICT0006']
value.append('hello')
print(value)

In [ ]:
d['S1/B04/B04_R1/S1_B04_R1_PICT0006']

In [ ]:
path2json = 'lila/SS_S1/SnapshotSerengetiBboxes_20190903.json'
coco = COCO(path2json)
ids = list(sorted(coco.imgs.keys()))
anns = list(coco.anns.keys())
d = dict.fromkeys(ids, [])
for i in anns:
    img_id = coco.anns[i]['image_id']
    value = d[img_id]
    value.append(i)
    #print(value)
    pair = {img_id: value}
    d.update(pair)

In [ ]:
id_dict = dict.frim
for i in ids[:10]:
    print(i)
    id_dict.update(i, )

In [ ]:
dict.fromkeys(keys)

In [ ]:
ann_ids = [i for i in anns if coco.anns[i]['image_id'] == img_id]

In [ ]:
coco.loadAnns(ann_ids)

In [ ]:
coco.imgs[img_id]

In [ ]:
ann_ids = list(coco.anns.keys())

In [ ]:
coco.anns[ann_ids[0]]

In [ ]:
img_ids = list(set([coco.anns[i]['image_id'] for i in ann_ids]))
#img_ids = [coco.anns[i]['image_id'] for i in ann_ids]

In [ ]:
img_ids = [(coco.anns[i]['image_id'], i) for i in ann_ids]

In [ ]:
coco.anns[ann_ids[0]]['image_id']

In [ ]:
img_ids = list(coco.imgs.keys())

In [ ]:
s1 = [i for i in img_ids if coco.imgs[i]['season'] == 'S1']
s2 = [i for i in img_ids if coco.imgs[i]['season'] == 'S2']
s3 = [i for i in img_ids if coco.imgs[i]['season'] == 'S3']
s4 = [i for i in img_ids if coco.imgs[i]['season'] == 'S4']
s5 = [i for i in img_ids if coco.imgs[i]['season'] == 'S5']
s6 = [i for i in img_ids if coco.imgs[i]['season'] == 'S6']

In [ ]:
print(len(s1))
print(len(s2))
print(len(s3))
print(len(s4))
print(len(s5))
print(len(s6))
print(len(img_ids))

In [ ]:
print(len(s1)+len(s2)+len(s3)+len(s4)+len(s5)+len(s6))

In [ ]:
len(img_ids)

In [ ]:
img_ids[0]

In [ ]:
coco.anns

In [ ]:
len(ids)

In [ ]:
ids[0]

In [ ]:
ids[0]

In [ ]:
img_id = ids[0]

In [ ]:
ann_ids = coco.getAnnIds(imgIds=img_id)

In [ ]:
target = coco.loadAnns(ann_ids)

In [ ]:
print(target)

In [ ]:
coco = COCO(path2json)

In [ ]:
cococ_train = CocoDetection(root = path2data, annFile = path2json)

In [ ]:
x = list(coco.anns.keys())[0]

In [ ]:
coco.anns[x]['bbox']

In [ ]:
import torchvision

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained_backbone=True)

In [ ]:
x = load_image('373_Ariel II right side_27Dec2014.jpg', resize_dim=(600, 1000))
x = torch.unsqueeze(x, 0)

In [ ]:
model.eval()
y = model(x)

In [ ]:
y

In [ ]:
%matplotlib inline
def show(img):
    img = img[0]
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

In [ ]:
show(x)

In [ ]:
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]

In [ ]:
model(x)